In [1]:
import scraper as s
import pandas as pd
import numpy as np
import datetime

In [3]:
m = s.stockMongo()
apple = m.get_options("AAPL")

In [4]:
apple

,ask,bid,contract,in-money,iv,last,last_trade,strike,strike-date,type,volume
date,,,,,,,,,,,
2020-06-09 04:56:55.325,0.00,0.00,AAPL210917C00110000,in-the-money,0.00%,197.67,2020-05-15,110.0,2021-09-17,Call,1.0
2020-06-09 04:56:55.325,147.45,138.05,AAPL210917C00120000,in-the-money,0.00%,122.30,2019-10-22,120.0,2021-09-17,Call,NaN
2020-06-09 04:56:55.325,209.50,204.50,AAPL210917C00125000,in-the-money,44.92%,179.31,2020-05-07,125.0,2021-09-17,Call,494.0
2020-06-09 04:56:55.325,0.00,0.00,AAPL210917C00130000,in-the-money,0.00%,201.75,2020-06-05,130.0,2021-09-17,Call,5.0
2020-06-09 04:56:55.325,178.60,175.30,AAPL210917C00135000,in-the-money,0.00%,136.69,2020-04-09,135.0,2021-09-17,Call,3.0
...,...,...,...,...,...,...,...,...,...,...,...
2020-08-01 20:01:40.807,70.25,67.50,AAPL200918P00490000,in-the-money,38.56%,98.05,2020-07-15,490.0,2020-09-18,Put,6.0
2020-08-01 20:01:40.807,127.05,124.10,AAPL200918P00495000,in-the-money,129.52%,132.20,2020-06-24,495.0,2020-09-18,Put,NaN
2020-08-01 20:01:40.807,114.75,113.05,AAPL200918P00500000,in-the-money,104.62%,177.56,2020-07-09,500.0,2020-09-18,Put,1.0


In [87]:
#apple = apple.where(apple['last_trade']==apple.index.date).dropna()
apple = apple.drop(['volume', 'iv', 'in-money', 'bid', 'ask','last_trade', 'contract'], axis=1)

In [88]:
apple = apple.reset_index()
apple = apple.set_index(['strike-date'])
apple['date'] = pd.to_datetime(apple['date']).dt.strftime('%m-%d-%Y')
apple['date'] = pd.to_datetime(apple['date'])


In [89]:
strike_pivot = apple.pivot_table(columns="strike-date", values="date", aggfunc=np.count_nonzero)
strike_cols = strike_pivot.columns

aCLo = apple.loc[('2020-09-18')]
apl = m.get_finnhub_prices('AAPL', min(aCLo['date']), max(aCLo['date']))
apl['date'] = apl.index
apl = apl.reset_index()
apl = apl.drop(['h', 'l', 'o', 'v','t'], axis=1)


In [63]:
strike_cols[0]
aCLo = apple.loc[(strike_cols[0])]

aCLo
strike_pivot2 = aCLo.pivot_table(columns="strike", values="date", aggfunc=np.count_nonzero)
strike_cols2 = strike_pivot2.columns
strike_cols2
apple.head()

,date,last,strike,type
strike-date,,,,
2021-09-17,2020-06-09,197.67,110.0,Call
2021-09-17,2020-06-09,122.30,120.0,Call
2021-09-17,2020-06-09,179.31,125.0,Call
2021-09-17,2020-06-09,201.75,130.0,Call
2021-09-17,2020-06-09,136.69,135.0,Call


In [68]:
new_apple = []
apple2 = apple
for cols in strike_cols:
    apple2 = apple2.reset_index()
    apple2 = apple2.set_index(['type','strike-date'])
    print(apple2.head())
    print(cols)
    #apple_pivot = apple2.loc[('Call', cols)]
    #strike_pivot = apple_pivot.pivot_table(columns="strike", values="date", aggfunc=np.count_nonzero)
    #strike_value_cols = strike_pivot.columns
    
    apple2 = apple2.set_index(['type', 'strike','strike-date'])
    for i in range(len(strike_value_cols)-1):
        aCLo = apple2.loc[('Call', strike_value_cols[i], cols)]
        aCLo = aCLo.reset_index()
        #aCLo = aCLo.drop(['type', 'strike-date'], axis=1)
        aCLo.rename(columns = {'last':'priceCLo', 'strike': 'strikeCLo'}, inplace = True)
        aCHi = apple2.loc[('Call', strike_value_cols[i+1], cols)]
        aCHi = aCHi.reset_index()
        #aCHi = aCHi.drop(['type', 'strike-date'], axis=1)
        aCHi.rename(columns = {'last':'priceCHi', 'strike': 'strikeCHi'}, inplace = True)
        aPLo = apple2.loc[('Put', strike_value_cols[i], cols)]
        aPLo = aPLo.reset_index()
        #aPLo = aPLo.drop(['type', 'strike-date'], axis=1)
        aPLo.rename(columns = {'last':'pricePLo', 'strike': 'strikePLo'}, inplace = True)
        aPHi = apple2.loc[('Put', strike_value_cols[i+1], cols)]
        aPHi = aPHi.reset_index()
        #aPHi = aPHi.drop(['type', 'strike-date'], axis=1)
        aPHi.rename(columns = {'last':'pricePHi', 'strike': 'strikePHi'}, inplace = True)
    
        a = pd.merge(aCLo, apl, on='date')
        a = pd.merge(a, aCHi, on='date')
        a = pd.merge(a, aPLo, on='date')
        a = pd.merge(a, aPHi, on='date')

        new_apple.append(a)

a[0]

  strike-date       date    last  strike  type
0  2021-09-17 2020-06-09  197.67   110.0  Call
1  2021-09-17 2020-06-09  122.30   120.0  Call
2  2021-09-17 2020-06-09  179.31   125.0  Call
3  2021-09-17 2020-06-09  201.75   130.0  Call
4  2021-09-17 2020-06-09  136.69   135.0  Call
2020-06-19 00:00:00


c:\userdata\z003rvhr\python\yahoo_graber\grab\lib\site-packages\ipykernel_launcher.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  


KeyError: 1.0

In [27]:
def iron_condor_price(x):
    s = -x['priceCHi']+x['priceCLo']+x['pricePHi']-x['pricePLo']
    
    return s/(x['strikePHi']-x['strikePLo'])

def iron_condor_value(x):
    s = 0
    if x['c'] >= x['strikeCLo']:
        s = s - x['c'] + x['strikeCLo']
    if x['c'] >= x['strikeCHi']:
        s = s + x['c'] - x['strikeCHi']
    if x['c'] <= x['strikePHi']:
        s = s - x['c'] + x['strikePHi']
    if x['c'] <= x['strikePLo']:
        s = s + x['c'] - x['strikePLo']
    #/(x['strikeP360']-x['strikeP350'])
    return s

In [28]:
a['daily_return'] = a.apply(iron_condor_price, axis=1).pct_change(1)
a['value'] = a.apply(iron_condor_value, axis=1)

In [29]:
a['daily_return'].describe()

count    18.000000
mean      0.054332
std       0.352774
min      -0.439853
25%      -0.174258
50%       0.002694
75%       0.140436
max       0.929167
Name: daily_return, dtype: float64

In [88]:
apple['time-left'] = apple['strike-date']-apple.index
apple[apple['time-left'].between(pd.Timedelta(1,'D'), pd.Timedelta(7,'D'))]

,last,strike,strike-date,type,time-left
date,,,,,
2020-06-12 18:01:24.609,4.74,327.5,2020-06-19,Put,6 days 05:58:35.391000
2020-06-12 18:01:24.609,5.60,330.0,2020-06-19,Put,6 days 05:58:35.391000
2020-06-12 18:01:24.609,6.55,332.5,2020-06-19,Put,6 days 05:58:35.391000
2020-06-12 18:01:24.609,7.50,335.0,2020-06-19,Put,6 days 05:58:35.391000
2020-06-12 18:01:24.609,8.73,337.5,2020-06-19,Put,6 days 05:58:35.391000
...,...,...,...,...,...
2020-07-15 20:01:35.342,139.25,515.0,2020-07-17,Put,1 days 03:58:24.658000
2020-07-15 20:01:35.342,120.60,520.0,2020-07-17,Put,1 days 03:58:24.658000
2020-07-15 20:01:35.342,131.70,530.0,2020-07-17,Put,1 days 03:58:24.658000


In [135]:
#apple = apple.reset_index()
#apple
b = apple.pivot_table(columns="strike-date", values="date", aggfunc=np.count_nonzero)
len(b.columns)


10

In [84]:
apple2 = apple2.reset_index()
apple2 = apple2.set_index(['type','strike-date'])
apple_pivot = apple2.loc[('Call', strike_cols[6])]

c:\userdata\z003rvhr\python\yahoo_graber\grab\lib\site-packages\ipykernel_launcher.py:3: PerformanceWarning: indexing past lexsort depth may impact performance.
  This is separate from the ipykernel package so we can avoid doing imports until


In [91]:
apple2

strike       date    last
type strike-date                           
Call 2021-09-17    110.0 2020-06-09  197.67
     2021-09-17    120.0 2020-06-09  122.30
     2021-09-17    125.0 2020-06-09  179.31
     2021-09-17    130.0 2020-06-09  201.75
     2021-09-17    135.0 2020-06-09  136.69
...                  ...        ...     ...
Put  2020-09-18    490.0 2020-08-01   98.05
     2020-09-18    495.0 2020-08-01  132.20
     2020-09-18    500.0 2020-08-01  177.56
     2020-09-18    545.0 2020-08-01  182.00
     2020-09-18    565.0 2020-08-01  154.36

[7231 rows x 3 columns]

In [106]:
apple2 = apple
apple2 = apple2.reset_index()
apple2 = apple2.set_index(['strike-date'])
strike_pivot = apple2.loc['Put'].pivot_table(columns="strike-date", values="date", aggfunc=np.count_nonzero)
strike_cols = strike_pivot.columns
strike_cols

DatetimeIndex(['2020-06-19', '2020-07-17', '2020-07-24', '2020-09-18',
               '2021-01-15', '2021-06-18', '2021-09-17', '2022-01-21',
               '2022-06-17', '2022-09-16'],
              dtype='datetime64[ns]', name='strike-date', freq=None)

In [109]:
apple2 = apple
apple2 = apple2.reset_index()
apple2 = apple2.set_index(['strike-date'])
apple2.loc['2020-06-19']

,date,ask,bid,contract,in-money,iv,last,last_trade,strike,type,volume
strike-date,,,,,,,,,,,
2020-06-19,2020-06-09 04:54:18.424,0.00,0.00,AAPL200619P00330000,in-the-money,1.56%,4.30,2020-06-08,330.0,Put,NaN
2020-06-19,2020-06-09 04:54:18.424,0.00,0.00,AAPL200619P00332500,in-the-money,0.39%,5.32,2020-06-08,332.5,Put,931.0
2020-06-19,2020-06-09 04:54:18.424,0.00,0.00,AAPL200619P00335000,in-the-money,0.00%,6.60,2020-06-08,335.0,Put,874.0
2020-06-19,2020-06-09 04:54:18.424,0.00,0.00,AAPL200619P00337500,in-the-money,0.00%,8.00,2020-06-08,337.5,Put,108.0
2020-06-19,2020-06-09 04:54:18.424,0.00,0.00,AAPL200619P00340000,in-the-money,0.00%,9.60,2020-06-08,340.0,Put,616.0
...,...,...,...,...,...,...,...,...,...,...,...
2020-06-19,2020-06-15 19:37:39.852,116.85,116.10,AAPL200619P00460000,in-the-money,0.00%,120.60,2020-06-12,460.0,Put,5.0
2020-06-19,2020-06-15 19:37:39.852,127.30,126.80,AAPL200619P00470000,in-the-money,0.00%,130.60,2020-06-12,470.0,Put,5.0
2020-06-19,2020-06-15 19:37:39.852,137.30,137.05,AAPL200619P00480000,in-the-money,0.00%,144.35,2020-06-15,480.0,Put,10.0
